In [18]:
import findspark
findspark.init()
findspark.find()

from IPython.display import *
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [19]:
import pyspark
print(pyspark.__version__)

3.5.2


In [20]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from delta import *

builder = SparkSession.builder.appName("DeltaLakeApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
   .config("spark.jars.packages", "io.delta:delta-core_2.12:3.2.0")
 


spark = configure_spark_with_delta_pip(builder).getOrCreate()
sc = spark.sparkContext


In [21]:
# Create schema for Yellow Taxi data
yellowTaxiSchema = (
                        StructType
                        ([ 
                            StructField("VendorId"               , IntegerType()   , True),
                            StructField("PickupTime"             , TimestampType() , True),
                            StructField("DropTime"               , TimestampType() , True),
                            StructField("PassengerCount"         , DoubleType()    , True),
                            StructField("TripDistance"           , DoubleType()    , True),
                            StructField("RateCodeId"             , DoubleType()    , True),
                            StructField("StoreAndFwdFlag"        , StringType()    , True),
                            StructField("PickupLocationId"       , IntegerType()   , True),
                            StructField("DropLocationId"         , IntegerType()   , True),
                            StructField("PaymentType"            , IntegerType()   , True),
                            StructField("FareAmount"             , DoubleType()    , True),
                            StructField("Extra"                  , DoubleType()    , True),
                            StructField("MtaTax"                 , DoubleType()    , True),
                            StructField("TipAmount"              , DoubleType()    , True),
                            StructField("TollsAmount"            , DoubleType()    , True),
                            StructField("ImprovementSurcharge"   , DoubleType()    , True),
                            StructField("TotalAmount"            , DoubleType()    , True),
                            StructField("CongestionSurcharge"    , DoubleType()    , True),
                            StructField("AirportFee"             , DoubleType()    , True)
                        ])
                   )


# Read Yellow Taxis file
yellowTaxiDF = (
                  spark
                    .read
                    .option("header", "true")    
                    .schema(yellowTaxiSchema)    
                    .csv(r"C:\Users\MonierAshraf\Desktop\2Btest\Raw\YellowTaxis_202210.csv")
               )


# Print schema of DataFrame
yellowTaxiDF.printSchema()

root
 |-- VendorId: integer (nullable = true)
 |-- PickupTime: timestamp (nullable = true)
 |-- DropTime: timestamp (nullable = true)
 |-- PassengerCount: double (nullable = true)
 |-- TripDistance: double (nullable = true)
 |-- RateCodeId: double (nullable = true)
 |-- StoreAndFwdFlag: string (nullable = true)
 |-- PickupLocationId: integer (nullable = true)
 |-- DropLocationId: integer (nullable = true)
 |-- PaymentType: integer (nullable = true)
 |-- FareAmount: double (nullable = true)
 |-- Extra: double (nullable = true)
 |-- MtaTax: double (nullable = true)
 |-- TipAmount: double (nullable = true)
 |-- TollsAmount: double (nullable = true)
 |-- ImprovementSurcharge: double (nullable = true)
 |-- TotalAmount: double (nullable = true)
 |-- CongestionSurcharge: double (nullable = true)
 |-- AirportFee: double (nullable = true)



### Create database in metastore

In [22]:
spark.sql("""CREATE DATABASE IF NOT EXISTS TaxisDB
""")

DataFrame[]

### Parquet format: Save DataFrame as a Table

In [23]:
(
    yellowTaxiDF
            .write    
            .mode("overwrite")
    
            .partitionBy("VendorId")
    
            .format("parquet")
    
            .option("path", "C:\Output\YellowTaxis.parquet")
    
            .saveAsTable("TaxisDB.YellowTaxisParquet")
)

### Delta format: Save DataFrame as Table

In [24]:
# If you have already run this command, and want to start from beginning,
# delete folder from file system first

from delta import *

(
    yellowTaxiDF
            .write    
            .mode("overwrite")
    
            .partitionBy("VendorId")
    
            .format("delta")
    
            .option("path", "C:\Output\YellowTaxis.delta")
    
            .saveAsTable("TaxisDB.YellowTaxis")
)

In [25]:
spark.sql("""

SELECT COUNT(*)
FROM TaxisDB.YellowTaxis

""").show()

+--------+
|count(1)|
+--------+
| 3675412|
+--------+



### Audit History of Delta Table

This shows transaction log of Delta Table

In [26]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                      |userMetadata|engineInfo                         |
+-------+-------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------

### Overwrite data in Delta table

In [27]:
# Overwrite data by re-running the command

(
    yellowTaxiDF
            .write    
            .mode("overwrite")
    
            .partitionBy("VendorId")
    
            .format("delta")
    
            .option("path", "C:\Output\YellowTaxis.delta")
    
            .saveAsTable("TaxisDB.YellowTaxis")
)

In [28]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                                      |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                      |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+---------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+----------

In [29]:
spark.sql("""

DROP TABLE TaxisDB.YellowTaxis

""")

DataFrame[]

### Create Table Definition

Create table using DDL command, and later add the data

In [31]:
spark.sql("""

CREATE TABLE TaxisDB.YellowTaxis
(
    VendorId                INT               COMMENT 'Vendor providing the ride',
    
    PickupTime              TIMESTAMP,
    DropTime                TIMESTAMP,
    
    PickupLocationId        INT               NOT NULL,
    DropLocationId          INT,
    
    PassengerCount          DOUBLE,
    TripDistance            DOUBLE,
    
    RateCodeId              DOUBLE,
    StoreAndFwdFlag         STRING,
    PaymentType             INT,
    
    FareAmount              DOUBLE,
    Extra                   DOUBLE,
    MtaTax                  DOUBLE,
    TipAmount               DOUBLE,
    TollsAmount             DOUBLE,
    ImprovementSurcharge    DOUBLE,
    TotalAmount             DOUBLE,
    CongestionSurcharge     DOUBLE,
    AirportFee              DOUBLE
)

USING DELTA                  -- default is Parquet

LOCATION "C:/Output/YellowTaxis.delta"

PARTITIONED BY (VendorId)    -- optional

COMMENT 'This table stores ride information for Yellow Taxis'

""")

DataFrame[]

In [32]:
spark.sql("""
DESCRIBE TABLE  EXTENDED TaxisDB.YellowTaxis

""").show(truncate=False)

+-----------------------+---------+-------------------------+
|col_name               |data_type|comment                  |
+-----------------------+---------+-------------------------+
|VendorId               |int      |Vendor providing the ride|
|PickupTime             |timestamp|NULL                     |
|DropTime               |timestamp|NULL                     |
|PickupLocationId       |int      |NULL                     |
|DropLocationId         |int      |NULL                     |
|PassengerCount         |double   |NULL                     |
|TripDistance           |double   |NULL                     |
|RateCodeId             |double   |NULL                     |
|StoreAndFwdFlag        |string   |NULL                     |
|PaymentType            |int      |NULL                     |
|FareAmount             |double   |NULL                     |
|Extra                  |double   |NULL                     |
|MtaTax                 |double   |NULL                     |
|TipAmou

In [33]:
spark.sql("""

DESCRIBE TABLE EXTENDED TaxisDB.YellowTaxis

""").show(53, truncate=False)

+----------------------------+---------------------------------------------------+-------------------------+
|col_name                    |data_type                                          |comment                  |
+----------------------------+---------------------------------------------------+-------------------------+
|VendorId                    |int                                                |Vendor providing the ride|
|PickupTime                  |timestamp                                          |NULL                     |
|DropTime                    |timestamp                                          |NULL                     |
|PickupLocationId            |int                                                |NULL                     |
|DropLocationId              |int                                                |NULL                     |
|PassengerCount              |double                                             |NULL                     |
|TripDistance      

### Options to Add Data to Delta Table

#### Option 1: Insert command

Use typical SQL Insert command to add data to table

In [34]:
spark.sql("""

INSERT INTO TaxisDB.YellowTaxis

-- (VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)

VALUES (3, '2022-12-01T00:00:00.000Z', '2022-12-01T00:15:34.000Z', 170, 140, 1.0, 2.9, 1.0, '1', 1, 13.0, 0.5, 0.5, 1.0, 0.0, 0.3, 15.3, 0.0, 0.0)

""")

DataFrame[]

In [35]:
spark.sql("""

SELECT * FROM TaxisDB.YellowTaxis

""").show(truncate=False)

+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|PickupTime         |DropTime           |PickupLocationId|DropLocationId|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|3       |2022-12-01 02:00:00|2022-12-01 02:15:34|170             |140           |1.0           |2.9         |1.0       |1              |1          |13.0      |0.5  |0.5   |1.0      |0.0        |0.3                 |15.3

In [36]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                     |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+

#### Option 2: Append a DataFrame

Read data as a DataFrame and append to Delta Table using PySpark

In [37]:
# Extract new records from Storage/Data Lake

yellowTaxisAppendDF = (
                          spark
                            .read
                            .option("header", "true")
                            .schema(yellowTaxiSchema)
                            .csv(r"C:\Users\MonierAshraf\Desktop\2Btest\Raw\YellowTaxis_append.csv")
                      )

yellowTaxisAppendDF.show()

+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PickupLocationId|DropLocationId|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 02:03:41|2022-10-01 02:18:39|           1.0|         1.7|       1.0|              N|             249|           107|          1|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [38]:
# Append to data lake in delta format

(
    yellowTaxisAppendDF
        .write
  
        .mode("append")
  
        .partitionBy("VendorId")  
        .format("delta")           
        .save("C:\Output\YellowTaxis.delta")
)

In [39]:
spark.sql("""

SELECT * 
FROM TaxisDB.YellowTaxis

""").show()

+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PickupLocationId|DropLocationId|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+----------------+--------------+--------------+------------+----------+---------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 02:03:41|2022-10-01 02:18:39|             249|           107|           1.0|         1.7|       1.0|              N|          1|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [40]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                     |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+

In [41]:
# Append to data lake in delta format
(
    yellowTaxiDF
        .write
  
        .mode("append")
  
        .partitionBy("VendorId")  
        .format("delta")
        .save("C:\Output\YellowTaxis.delta")
)

In [42]:
spark.sql("""

SELECT COUNT(1)
FROM TaxisDB.YellowTaxis

""").show()

+--------+
|count(1)|
+--------+
| 3675416|
+--------+



### Check files holding the data

In [43]:
spark.sql("""

SELECT INPUT_FILE_NAME()
     
     , VendorId
     , PickupLocationId
     , PassengerCount
     
FROM TaxisDB.YellowTaxis

WHERE VendorId = 3

""").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------+--------+----------------+--------------+
|input_file_name()                                                                                               |VendorId|PickupLocationId|PassengerCount|
+----------------------------------------------------------------------------------------------------------------+--------+----------------+--------------+
|file:/C:/Output/YellowTaxis.delta/VendorId=3/part-00000-6a4a84b6-5603-4be7-9d7d-7dfb84f8deee.c000.snappy.parquet|3       |249             |1.0           |
|file:/C:/Output/YellowTaxis.delta/VendorId=3/part-00000-6a4a84b6-5603-4be7-9d7d-7dfb84f8deee.c000.snappy.parquet|3       |151             |2.0           |
|file:/C:/Output/YellowTaxis.delta/VendorId=3/part-00000-6a4a84b6-5603-4be7-9d7d-7dfb84f8deee.c000.snappy.parquet|3       |238             |1.0           |
|file:/C:/Output/YellowTaxis.delta/VendorId=3/part-00000-01a86f7

### UPDATE command

In [44]:
# Check passenger count

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 249

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
|       3|             249|           1.0|
+--------+----------------+--------------+



In [45]:
# Run update statement to change passenger count

spark.sql("""

UPDATE TaxisDB.YellowTaxis

SET PassengerCount = 2

WHERE VendorId = 3
    AND PickupLocationId = 249

""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [46]:
# Check passenger count after update

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 249

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
|       3|             249|           2.0|
+--------+----------------+--------------+



In [47]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                     |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                         

### DELETE command

In [48]:
# Check if record exist

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 151

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
|       3|             151|           2.0|
+--------+----------------+--------------+



In [49]:
# Delete the record

spark.sql("""

DELETE FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 151

""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [50]:
# Check if record exist after delete operation

spark.sql("""

SELECT VendorId
     , PickupLocationId
     , PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 151

""").show()

+--------+----------------+--------------+
|VendorId|PickupLocationId|PassengerCount|
+--------+----------------+--------------+
+--------+----------------+--------------+



In [51]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation   |operationParameters                                                                                                                                     |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                         

### MERGE command

In [52]:
# Extract changed records from Storage/Data Lake

yellowTaxiChangesDF = (
                          spark
                            .read
                            .option("header", "true")
                            .schema(yellowTaxiSchema)
                            .csv(r"C:\Users\MonierAshraf\Desktop\2Btest\Raw\YellowTaxis_changes.csv")
                      )

yellowTaxiChangesDF.show()

+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PickupLocationId|DropLocationId|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 02:03:41|2022-10-01 02:18:39|           1.0|         1.7|       1.0|              N|             249|           107|          3|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [53]:
yellowTaxiChangesDF.createOrReplaceTempView("YellowTaxiChanges")

In [54]:
spark.sql("""

SELECT *
FROM YellowTaxiChanges

""").show()

+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|VendorId|         PickupTime|           DropTime|PassengerCount|TripDistance|RateCodeId|StoreAndFwdFlag|PickupLocationId|DropLocationId|PaymentType|FareAmount|Extra|MtaTax|TipAmount|TollsAmount|ImprovementSurcharge|TotalAmount|CongestionSurcharge|AirportFee|
+--------+-------------------+-------------------+--------------+------------+----------+---------------+----------------+--------------+-----------+----------+-----+------+---------+-----------+--------------------+-----------+-------------------+----------+
|       3|2022-10-01 02:03:41|2022-10-01 02:18:39|           1.0|         1.7|       1.0|              N|             249|           107|          3|       9.5|  3.0|   0.5|     2.65|        0.0|                 0.3|    

In [55]:
spark.sql("""

MERGE INTO TaxisDB.YellowTaxis tgt

    USING YellowTaxiChanges    src

        ON    tgt.VendorId          =  src.VendorId
          AND tgt.PickupLocationId  =  src.PickupLocationId
  
-- Update row if join conditions match
WHEN MATCHED
      
      THEN  
          UPDATE SET    tgt.PaymentType = src.PaymentType   
          
                                                      -- Use 'UPDATE SET *' to update all columns

-- Insert row if row is not present in target table
WHEN NOT MATCHED 
      AND PickupTime >= '2022-03-01'

      THEN 
          INSERT (VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, 
                  RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, 
                  ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)
          
          VALUES (VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance,
                  RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, 
                  ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)
                                                                         


""").show()

+-----------------+----------------+----------------+-----------------+
|num_affected_rows|num_updated_rows|num_deleted_rows|num_inserted_rows|
+-----------------+----------------+----------------+-----------------+
|                4|               3|               0|                1|
+-----------------+----------------+----------------+-----------------+



In [56]:
spark.sql(""" 
DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Constraints on Delta Table

#### NOT NULL constraint

Drop the constraint here which was previously created

<i>- Define NOT NULL constraint on column to avoid insertion of NULL values </i> <br/>

In [64]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

      CHANGE COLUMN PickupLocationId SET NOT NULL

""").show()

AnalysisException: Cannot change nullable column to non-nullable: PickupLocationId.; line 3 pos 0;
AlterColumn resolvedfieldname(StructField(PickupLocationId,IntegerType,true)), false
+- ResolvedTable org.apache.spark.sql.delta.catalog.DeltaCatalog@38fc9810, TaxisDB.YellowTaxis, DeltaTableV2(org.apache.spark.sql.SparkSession@49b156cf,file:/C:/Output/YellowTaxis.delta,Some(CatalogTable(
Catalog: spark_catalog
Database: taxisdb
Table: yellowtaxis
Created Time: Sat Nov 16 00:31:47 EET 2024
Last Access: UNKNOWN
Created By: Spark 3.5.2
Type: EXTERNAL
Provider: delta
Comment: This table stores ride information for Yellow Taxis
Location: file:/C:/Output/YellowTaxis.delta
Partition Provider: Catalog)),Some(TaxisDB.YellowTaxis),None,Map()), [VendorId#20031, PickupTime#20032, DropTime#20033, PickupLocationId#20034, DropLocationId#20035, PassengerCount#20036, TripDistance#20037, RateCodeId#20038, StoreAndFwdFlag#20039, PaymentType#20040, FareAmount#20041, Extra#20042, MtaTax#20043, TipAmount#2004

In [59]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [70]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

      CHANGE COLUMN PickupLocationId DROP NOT NULL

""").show()

# ALTER TABLE TaxisDB.YellowTaxis
#      CHANGE COLUMN PickupLocationId SET NOT NULL

++
||
++
++



In [71]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#### CHECK constraint

Check constraint helps to enforce certain conditions on the table

In [72]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

    ADD CONSTRAINT PassengerCountCheck CHECK (PassengerCount <= 5)

""").show()

AnalysisException: [DELTA_NEW_CHECK_CONSTRAINT_VIOLATION] 170881 rows in spark_catalog.taxisdb.yellowtaxis violate the new CHECK constraint (PassengerCount <= 5)

In [73]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

    ADD CONSTRAINT PassengerCheck CHECK (PassengerCount <= 9 OR PassengerCount IS NULL)

""")

DataFrame[]

In [74]:
spark.sql("""

INSERT INTO TaxisDB.YellowTaxis

--(VendorId, PickupTime, DropTime, PickupLocationId, DropLocationId, PassengerCount, TripDistance, RateCodeId, StoreAndFwdFlag, PaymentType, FareAmount, Extra, MtaTax, TipAmount, TollsAmount, ImprovementSurcharge, TotalAmount, CongestionSurcharge, AirportFee)

VALUES (3, '2022-12-01T00:00:00.000Z', '2022-12-01T00:15:34.000Z', 170, 140, 

        20,  -- PassengerCount

        2.9, 1.0, '1', 1, 13.0, 0.5, 0.5, 1.0, 0.0, 0.3, 15.3, 0.0, 0.0)

""")

Py4JJavaError: An error occurred while calling o33.sql.
: org.apache.spark.sql.delta.schema.DeltaInvariantViolationException: [DELTA_VIOLATE_CONSTRAINT_WITH_VALUES] CHECK constraint passengercheck ((PassengerCount <= 9) OR (PassengerCount IS NULL)) violated by row with values:
 - PassengerCount : 20.0
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.getConstraintViolationWithValuesException(InvariantViolationException.scala:75)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.apply(InvariantViolationException.scala:101)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException$.apply(InvariantViolationException.scala:112)
	at org.apache.spark.sql.delta.schema.DeltaInvariantViolationException.apply(InvariantViolationException.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.delta.constraints.DeltaInvariantCheckerExec.$anonfun$doExecu

In [75]:
spark.sql("""

ALTER TABLE TaxisDB.YellowTaxis

    DROP CONSTRAINT PassengerCheck

""")

DataFrame[]

In [76]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Update a record

In [77]:
# Check passenger count before update

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show()

+--------------+
|PassengerCount|
+--------------+
|           0.0|
+--------------+



In [78]:
# Update passenger count

spark.sql("""

UPDATE TaxisDB.YellowTaxis

SET PassengerCount = 1

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show()

+-----------------+
|num_affected_rows|
+-----------------+
|                1|
+-----------------+



In [79]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Time Travel: Access using Version Number

In [80]:
# Check at initial version

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis        VERSION AS OF 0

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show(truncate=False)

+--------------+
|PassengerCount|
+--------------+
+--------------+



In [64]:
# Check at one prior version

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis        VERSION AS OF 6

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show(truncate=False)

+--------------+
|PassengerCount|
+--------------+
|0.0           |
+--------------+



### Time Travel: Access using Timestamp

In [82]:
spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis        TIMESTAMP AS OF '2024-11-16 00:32:37.744'

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show(truncate=False)

+--------------+
|PassengerCount|
+--------------+
|0.0           |
+--------------+



### Restore Table to older version

In [83]:
spark.sql("""

RESTORE TABLE TaxisDB.YellowTaxis    TO VERSION AS OF 6

""").show(truncate=False)

+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|table_size_after_restore|num_of_files_after_restore|num_removed_files|num_restored_files|removed_files_size|restored_files_size|
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+
|81727873                |34                        |1                |1                 |5637              |5637               |
+------------------------+--------------------------+-----------------+------------------+------------------+-------------------+



In [84]:
spark.sql("""

DESCRIBE HISTORY TaxisDB.YellowTaxis

""").show(truncate=False)

+-------+-----------------------+------+--------+---------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [85]:
# Check data after restore

spark.sql("""

SELECT PassengerCount

FROM TaxisDB.YellowTaxis

WHERE VendorId = 3
    AND PickupLocationId = 1

""").show()

+--------------+
|PassengerCount|
+--------------+
|           0.0|
+--------------+

